In [ ]:
# For python modules,
# Use in dev repository

In [1]:
from crimson.file_loader import collect_files

In [2]:
def my_editor(text:str)->str:
    text = text.replace('..%', '')
    return text

In [10]:
collect_files(
    '../',
    './flatten_module',
    includes=['src/', 'test/'],
    excludes=['egg-info', 'cache'],
    post_path_editor=my_editor
)

Files collected from ./ to ./flatten_module


In [1]:
import os
import nbformat
from nbformat.reader import NotJSONError

def convert_ipynb_to_py(root_dir: str, output_dir: str, editor=lambda x: x):
    """
    Convert .ipynb files under `root_dir` to .py files in `output_dir`.

    Args:
        root_dir (str): Input directory containing .ipynb files
        output_dir (str): Target directory to save .py files
        editor (Callable[[str], str]): Function to edit file name (without extension)
    """
    for dirpath, _, filenames in os.walk(root_dir):
        for fname in filenames:
            if not fname.endswith(".ipynb"):
                continue

            full_input_path = os.path.join(dirpath, fname)
            rel_path = os.path.relpath(full_input_path, root_dir)
            rel_dir = os.path.dirname(rel_path)
            base_name = os.path.splitext(os.path.basename(fname))[0]

            edited_name = editor(base_name) + ".py"
            target_dir = os.path.join(output_dir, rel_dir)
            os.makedirs(target_dir, exist_ok=True)
            full_output_path = os.path.join(target_dir, edited_name)

            try:
                nb = nbformat.read(full_input_path, as_version=4)
                code_cells = [
                    cell.source for cell in nb.cells if cell.cell_type == "code"
                ]
                with open(full_output_path, "w", encoding="utf-8") as f:
                    f.write("# Converted from: " + fname + "\n\n")
                    f.write("\n\n".join(code_cells))
                print(f"[✔] Converted: {rel_path} → {edited_name}")
            except NotJSONError:
                print(f"[✘] Skipped invalid notebook: {rel_path}")


In [ ]:
def editor(text: str) -> str:
	# Example editor function to modify the file name
	return "test_" + text.replace(".ipynb", ".py")

convert_ipynb_to_py(root_dir="./example", output_dir="./test",editor=editor)

[✔] Converted: control/number.ipynb → test_number.py
[✔] Converted: addon/pydantic_compatible.ipynb → test_pydantic_compatible.py
[✔] Converted: addon/packs.ipynb → test_packs.py
[✔] Converted: addon/intelli_type.ipynb → test_intelli_type.py
[✔] Converted: addon/annotated.ipynb → test_annotated.py


In [3]:
import os
import subprocess

def format_py_files_with_black(directory: str):
    """
    Apply Black formatting to all .py files in the given directory (non-recursive).

    Args:
        directory (str): Path to the directory containing Python files
    """
    files = [
        os.path.join(directory, f)
        for f in os.listdir(directory)
        if f.endswith(".py") and os.path.isfile(os.path.join(directory, f))
    ]

    if not files:
        print("[ℹ] No .py files found in:", directory)
        return

    try:
        subprocess.run(["black", *files], check=True)
        print(f"[✔] Formatted {len(files)} file(s) with Black in '{directory}'")
    except subprocess.CalledProcessError as e:
        print(f"[✘] Black formatting failed: {e}")


In [6]:
format_py_files_with_black("./test/addon")

[✔] Formatted 4 file(s) with Black in './test/addon'


reformatted test/addon/test_annotated.py
reformatted test/addon/test_pydantic_compatible.py

All done! ✨ 🍰 ✨
2 files reformatted, 2 files left unchanged.


In [4]:
from black import format_file_in_place, FileMode, WriteBack
from pathlib import Path
import os

def format_file(path: str):
    """
    Format a single Python file using Black's Python API.
    """
    file_path = Path(path)
    if not file_path.exists() or not file_path.suffix == ".py":
        print(f"[!] Skipped non-.py file: {file_path}")
        return

    changed = format_file_in_place(
        file_path,
        fast=False,
        mode=FileMode(),
        write_back=WriteBack.YES
    )
    if changed:
        print(f"[✔] Formatted: {file_path}")
    else:
        print(f"[✓] Already formatted: {file_path}")


In [5]:
def format_all_py_files_in_dir(directory: str):
    for fname in os.listdir(directory):
        full_path = os.path.join(directory, fname)
        if fname.endswith(".py") and os.path.isfile(full_path):
            format_file(full_path)


In [9]:
from black import format_file_in_place, FileMode, WriteBack
from pathlib import Path

def format_all_py_files_recursive(root_dir: str):
    """
    Recursively apply Black formatting to all .py files under `root_dir`.

    Args:
        root_dir (str): Root directory to walk through
    """
    root = Path(root_dir)
    count = 0

    for file_path in root.rglob("*.py"):  # <-- 핵심: 재귀적으로 .py 파일 찾기
        if file_path.is_file():
            changed = format_file_in_place(
                file_path,
                fast=False,
                mode=FileMode(),
                write_back=WriteBack.YES
            )
            msg = "✔ Formatted" if changed else "✓ Already formatted"
            print(f"[{msg}] {file_path}")
            count += 1

    print(f"\n✅ Total files processed: {count}")


In [10]:
format_all_py_files_recursive("./test")

[✔ Formatted] test/control/test_number.py
[✓ Already formatted] test/addon/test_pydantic_compatible.py
[✓ Already formatted] test/addon/test_annotated.py
[✓ Already formatted] test/addon/test_intelli_type.py
[✓ Already formatted] test/addon/test_packs.py

✅ Total files processed: 5


In [11]:
from black import format_file_in_place, FileMode, WriteBack
from pathlib import Path
import subprocess

def format_and_lint_py_files(root_dir: str):
    """
    Format all .py files under root_dir using Black,
    and report syntax/lint errors using flake8.
    
    Returns:
        {
            "formatted": [...],      # files formatted by Black
            "unchanged": [...],      # files untouched by Black
            "errors": { file: [msgs] }  # files with lint/syntax issues
        }
    """
    root = Path(root_dir)
    formatted, unchanged, errors = [], [], {}

    for file_path in root.rglob("*.py"):
        if file_path.is_file():
            changed = format_file_in_place(
                file_path,
                fast=False,
                mode=FileMode(),
                write_back=WriteBack.YES
            )
            (formatted if changed else unchanged).append(str(file_path))

            # Run flake8 on each file
            try:
                result = subprocess.run(
                    ["flake8", str(file_path)],
                    capture_output=True,
                    text=True,
                    check=False
                )
                if result.stdout:
                    errors[str(file_path)] = result.stdout.strip().split("\n")
            except Exception as e:
                errors[str(file_path)] = [f"[flake8 failed] {e}"]

    return {
        "formatted": formatted,
        "unchanged": unchanged,
        "errors": errors,
    }


In [13]:
format_and_lint_py_files("./test")

{'formatted': [],
 'unchanged': ['test/control/test_number.py',
  'test/addon/test_pydantic_compatible.py',
  'test/addon/test_annotated.py',
  'test/addon/test_intelli_type.py',
  'test/addon/test_packs.py'],
 'errors': {'test/control/test_number.py': ["test/control/test_number.py:15:38: F722 syntax error in forward annotation '(0,1]]'"],
  'test/addon/test_annotated.py': ["test/addon/test_annotated.py:31:26: F722 syntax error in forward annotation 'Any Meta Data'"]}}